In [6]:
from datapipeline import DataFeed
self = DataFeed()
brazil = self.get_BrazilCFR
seasia = self.get_SEAsiaCFR
foodpriceindex = self.get_food_price_index
eurusd = self.get_eurusd
cpi = self.get_g20_cpi
gdp = self.get_gdp
netback = self.get_historical_netback
minenetback = self.get_historical_mineNetback
naturalgas = self.get_natural_gas
ethanol = self.get_ethanol
fertilizerprod = self.get_total_fertilizer_production
freightcost = self.get_freightCost
interim = self.get_interimPricing

In [7]:
brazil.dropna(inplace=True)
seasia.dropna(inplace=True)
foodpriceindex.dropna(inplace=True)
eurusd.dropna(inplace=True)
cpi.dropna(inplace=True)
gdp.dropna(inplace=True)
netback.dropna(inplace=True)
minenetback.dropna(inplace=True)
naturalgas.dropna(inplace=True)
ethanol.dropna(inplace=True)
fertilizerprod.dropna(inplace=True)
freightcost.dropna(inplace=True)
interim.dropna(inplace=True)

In [8]:
dataset = [brazil,seasia,foodpriceindex,eurusd,cpi,gdp,netback,minenetback,naturalgas,ethanol,fertilizerprod,freightcost,interim]

In [9]:
[len(i) for i in dataset]

[164, 164, 382, 282, 266, 145, 103, 158, 187, 218, 116, 173, 92]

In [5]:
fertilizerprod

,Total Fertilizer Production,FertProdQuad,std_quad,std_1
Year,,,,
2020-12-01,263764,69571447696,3.871969e+09,7614.058017
2021-01-01,266756,71158763536,3.880871e+09,7624.433910
2021-02-01,266756,71158763536,3.880037e+09,7615.832303
2021-03-01,266756,71158763536,3.869462e+09,7588.188660
2021-04-01,266756,71158763536,3.849064e+09,7541.293580
2021-05-01,266756,71158763536,3.818688e+09,7474.784736
2021-06-01,266756,71158763536,3.778091e+09,7388.132450
2021-07-01,266756,71158763536,3.726940e+09,7280.617529
2021-08-01,266756,71158763536,3.664793e+09,7151.299068


In [ ]:
[len(i) for i in dataset]

In [ ]:
dataset[8]